# Natural Language Processing with Disaster Tweets

This is the link of the competition: https://www.kaggle.com/competitions/nlp-getting-started/overview

In [27]:
# Importing libraries
import pandas as pd
import string
from nltk.corpus import stopwords 
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

# scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [7]:
# This is the sample format way to submit the contest
file_loc = 'data/sample_submission.csv'
df = pd.read_csv(file_loc)
df.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [8]:
file_loc = 'data/train.csv'
df = pd.read_csv(file_loc)

In [9]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [3]:
stop_w = set(stopwords.words('english'))

def preprocessing(sentence):
    
    wl = WordNetLemmatizer()
    sentence = sentence.strip()
    sentence = sentence.lower()
    list_word = sentence.split()
    list_word = [wl.lemmatize(''.join(
                    [
                        c for c in char if not c.isdigit() 
                          and c not in string.punctuation

                ]               
            ), pos='v' )
        for char in list_word
        if char not in stop_w
        ]
    
    return ' '.join(list_word)

In [14]:
df['clean_text'] = [preprocessing(s) for s in df['text']]

In [16]:
df

,id,keyword,location,text,target,clean_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deeds reason earthquake may allah forgive us
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,residents ask shelter place notify officer eva...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfires evacuation order cal...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,get send photo ruby alaska smoke wildfires pou...
...,...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1,two giant crane hold bridge collapse nearby ho...
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1,ariaahrary thetawniest control wild fire calif...
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,m utckm volcano hawaii httptcozdtoydebj
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1,police investigate ebike collide car little po...


In [24]:
# The vectorizer model
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)

# Transform the text data to feature vectors
X = vectorizer.fit_transform(df['text'])
y = df['target']

In [25]:


# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize and train the classifier
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

SVC(kernel='linear')

In [28]:
# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate the performance
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names= newsgroups.target_names)

print(f'Accuracy: {accuracy:.4f}')
print('Classification Report:')
print(report)

NameError: name 'newsgroups' is not defined